# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-10 16:35:20.468574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-10 16:35:20.468596: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'pen_digits'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[ 0.23895455,  0.91749192, -0.51650341, ...,  1.66082614,
        -0.17451493,  1.93296504],
       [-1.13305889,  0.23922621, -0.51650341, ..., -0.9934575 ,
         1.26231828, -0.63855782],
       [-1.13305889, -1.73391041, -0.36465342, ..., -0.29972428,
        -0.74924822, -0.80626583],
       ...,
       [ 0.50168052,  0.91749192, -0.51650341, ...,  1.75131309,
        -0.22240937,  1.79320836],
       [-0.57841516,  0.91749192, -1.54149087, ...,  1.87196234,
        -0.89293154,  1.45779234],
       [-0.02377143,  0.91749192, -0.13687843, ..., -0.26956196,
         0.42416557, -0.80626583]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res  
    
    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)
    
    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))
    

## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 9937, 1: 1055})
Ratio-> 9.4 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Classifier:  XGBClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[1986    2]
 [  10  201]]

TNR: 0.99899
TPR: 0.95261
TNRxTPR: 0.95165
G-mean: 0.97552
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   2  209]]

TNR: 1.0
TPR: 0.99052
TNRxTPR: 0.99052
G-mean: 0.99525
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [10]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe13f0951f0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [11]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({1: 7949, 0: 7949})
Ratio->  1 : 1.0


In [12]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1987    1]
 [   1  210]]

TNR: 0.9995
TPR: 0.99526
TNRxTPR: 0.99476
G-mean: 0.99738
__________________________________________________
Classifier:  XGBClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[1985    3]
 [   7  204]]

TNR: 0.99849
TPR: 0.96682
TNRxTPR: 0.96537
G-mean: 0.98253
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   2  209]]

TNR: 1.0
TPR: 0.99052
TNRxTPR: 0.99052
G-mean: 0.99525
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [13]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [14]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({1: 7949, 0: 7949})
Ratio->  1 : 1.0


In [15]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Classifier:  XGBClassifier
[[1987    1]
 [   0  211]]

TNR: 0.9995
TPR: 1.0
TNRxTPR: 0.9995
G-mean: 0.99975
__________________________________________________
Classifier:  BaggingClassifier
[[1986    2]
 [   9  202]]

TNR: 0.99899
TPR: 0.95735
TNRxTPR: 0.95638
G-mean: 0.97795
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [16]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-09 23:24:39.494680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-09 23:24:39.494706: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-09 23:24:39.494721: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-09 23:24:39.494929: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({1.0: 7949, 0.0: 7949})
Ratio->  1 : 1.0


In [18]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1987    1]
 [   2  209]]

TNR: 0.9995
TPR: 0.99052
TNRxTPR: 0.99002
G-mean: 0.995
__________________________________________________
Classifier:  XGBClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Classifier:  BaggingClassifier
[[1987    1]
 [   9  202]]

TNR: 0.9995
TPR: 0.95735
TNRxTPR: 0.95686
G-mean: 0.97819
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   2  209]]

TNR: 1.0
TPR: 0.99052
TNRxTPR: 0.99052
G-mean: 0.99525
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [22]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=.9) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe18682f700>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [23]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.8)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 7154, 0: 6359})
Ratio->  1 : 1.1


In [24]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Classifier:  XGBClassifier
[[1987    1]
 [   0  211]]

TNR: 0.9995
TPR: 1.0
TNRxTPR: 0.9995
G-mean: 0.99975
__________________________________________________
Classifier:  BaggingClassifier
[[1988    0]
 [   6  205]]

TNR: 1.0
TPR: 0.97156
TNRxTPR: 0.97156
G-mean: 0.98568
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   1  210]]

TNR: 1.0
TPR: 0.99526
TNRxTPR: 0.99526
G-mean: 0.99763
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [25]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [26]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.9)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 7949, 0: 7154})
Ratio->  1 : 1.1


In [27]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1986    2]
 [   1  210]]

TNR: 0.99899
TPR: 0.99526
TNRxTPR: 0.99426
G-mean: 0.99713
__________________________________________________
Classifier:  XGBClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[1985    3]
 [   9  202]]

TNR: 0.99849
TPR: 0.95735
TNRxTPR: 0.9559
G-mean: 0.9777
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   3  208]]

TNR: 1.0
TPR: 0.98578
TNRxTPR: 0.98578
G-mean: 0.99287
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [28]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=.9)

In [29]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.7)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 7238, 0.0: 5564})
Ratio->  1 : 1.3


In [30]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[1987    1]
 [   0  211]]

TNR: 0.9995
TPR: 1.0
TNRxTPR: 0.9995
G-mean: 0.99975
__________________________________________________
Classifier:  XGBClassifier
[[1987    1]
 [   0  211]]

TNR: 0.9995
TPR: 1.0
TNRxTPR: 0.9995
G-mean: 0.99975
__________________________________________________
Classifier:  BaggingClassifier
[[1986    2]
 [   8  203]]

TNR: 0.99899
TPR: 0.96209
TNRxTPR: 0.96112
G-mean: 0.98037
__________________________________________________
Classifier:  RandomForestClassifier
[[1988    0]
 [   2  209]]

TNR: 1.0
TPR: 0.99052
TNRxTPR: 0.99052
G-mean: 0.99525
__________________________________________________
Ensemble predictions (majority voting):
[[1988    0]
 [   0  211]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
